# Intro

### Global variables

In [1]:
DATASET_PATH = "/home/jbonastre/tfm/dataset/AML-Cytomorphology"
SPLIT_BALANCED_PATH = "/home/jbonastre/tfm/dataset/DatasetSplitBalanced"

In [2]:
num_classes = 15
SHAPE_SIDE = 144

In [3]:
LABELS = ["BAS","EBO","EOS","KSC","LYA","LYT","MMZ","MOB","MON","MYB","MYO","NGB","NGS","PMB","PMO"]
CODE_TO_LABEL = {
    0:"BAS",
    1:"EBO",
    2:"EOS",
    3:"KSC",
    4:"LYA",
    5:"LYT",
    6:"MMZ",
    7:"MOB",
    8:"MON",
    9:"MYB",
    10:"MYO",
    11:"NGB",
    12:"NGS",
    13:"PMB",
    14:"PMO",
}
LABEL_TO_CODE = {
    "BAS":0,
    "EBO":1,
    "EOS":2,
    "KSC":3,
    "LYA":4,
    "LYT":5,
    "MMZ":6,
    "MOB":7,
    "MON":8,
    "MYB":9,
    "MYO":10,
    "NGB":11,
    "NGS":12,
    "PMB":13,
    "PMO":14,
}
LABELS_ABREVIATIONS = {
    "BAS": "Basophil",
    "EBO": "Erythroblast",
    "EOS": "Eosinophil",
    "KSC": "Smudge cell",
    "LYA": "Lymphocyte (atypical)",
    "LYT": "Lymphocyte (typical)",
    "MMZ": "Metamyelocyte",
    "MOB": "Monoblast",
    "MON": "Monocyte",
    "MYB": "Myelocyte",
    "MYO": "Myeloblast",
    "NGB": "Neutrophil (band)",
    "NGS": "Neutrophil (segmented)",
    "PMB": "Promyelocyte (bilobled)",
    "PMO": "Promyelocyte",
}

### Imports

In [4]:
import pickle
import numpy as np
import keras
import os
import PIL
import tensorflow as tf
from matplotlib import pyplot as plt
import cv2
import random
import math

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from keras.utils.np_utils import to_categorical

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

from os import path
import pickle

from collections import Counter


2023-07-10 19:38:16.946088: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Read images

In [5]:
labels_text = os.listdir(DATASET_PATH)
images = [[] for i in range(num_classes)]
N_SAMPLES_PER_LABEL = [0 for i in range(num_classes)]

for i,label_text in enumerate(labels_text):
    label_code = LABEL_TO_CODE[label_text]
    label_path = os.path.join(DATASET_PATH, label_text)
    label_images = os.listdir(label_path)
    N_SAMPLES_PER_LABEL[label_code] = len(label_images)
    for j,img_text in enumerate(label_images):
        print(f"[{i}] {label_text}: {j}. {img_text}")
        img_path = os.path.join(DATASET_PATH,label_text,img_text)
        img = np.array(PIL.Image.open(img_path))[:,:,:3]
        img = cv2.resize(img, (SHAPE_SIDE, SHAPE_SIDE))
        images[label_code].append(img)

[0] EOS: 0. EOS_0326.tiff
[0] EOS: 1. EOS_0181.tiff
[0] EOS: 2. EOS_0022.tiff
[0] EOS: 3. EOS_0303.tiff
[0] EOS: 4. EOS_0117.tiff
[0] EOS: 5. EOS_0158.tiff
[0] EOS: 6. EOS_0003.tiff
[0] EOS: 7. EOS_0406.tiff
[0] EOS: 8. EOS_0313.tiff
[0] EOS: 9. EOS_0234.tiff
[0] EOS: 10. EOS_0244.tiff
[0] EOS: 11. EOS_0110.tiff
[0] EOS: 12. EOS_0065.tiff
[0] EOS: 13. EOS_0413.tiff
[0] EOS: 14. EOS_0119.tiff
[0] EOS: 15. EOS_0261.tiff
[0] EOS: 16. EOS_0411.tiff
[0] EOS: 17. EOS_0398.tiff
[0] EOS: 18. EOS_0177.tiff
[0] EOS: 19. EOS_0133.tiff
[0] EOS: 20. EOS_0407.tiff
[0] EOS: 21. EOS_0222.tiff
[0] EOS: 22. EOS_0342.tiff
[0] EOS: 23. EOS_0315.tiff
[0] EOS: 24. EOS_0029.tiff
[0] EOS: 25. EOS_0298.tiff
[0] EOS: 26. EOS_0235.tiff
[0] EOS: 27. EOS_0128.tiff
[0] EOS: 28. EOS_0305.tiff
[0] EOS: 29. EOS_0078.tiff
[0] EOS: 30. EOS_0190.tiff
[0] EOS: 31. EOS_0013.tiff
[0] EOS: 32. EOS_0229.tiff
[0] EOS: 33. EOS_0260.tiff
[0] EOS: 34. EOS_0246.tiff
[0] EOS: 35. EOS_0259.tiff
[0] EOS: 36. EOS_0083.tiff
[0] EOS: 37

[0] EOS: 410. EOS_0213.tiff
[0] EOS: 411. EOS_0141.tiff
[0] EOS: 412. EOS_0018.tiff
[0] EOS: 413. EOS_0184.tiff
[0] EOS: 414. EOS_0266.tiff
[0] EOS: 415. EOS_0049.tiff
[0] EOS: 416. EOS_0106.tiff
[0] EOS: 417. EOS_0075.tiff
[0] EOS: 418. EOS_0051.tiff
[0] EOS: 419. EOS_0152.tiff
[0] EOS: 420. EOS_0113.tiff
[0] EOS: 421. EOS_0380.tiff
[0] EOS: 422. EOS_0166.tiff
[0] EOS: 423. EOS_0248.tiff
[1] MOB: 0. MOB_0001.tiff
[1] MOB: 1. MOB_0005.tiff
[1] MOB: 2. MOB_0021.tiff
[1] MOB: 3. MOB_0014.tiff
[1] MOB: 4. MOB_0019.tiff
[1] MOB: 5. MOB_0002.tiff
[1] MOB: 6. MOB_0004.tiff
[1] MOB: 7. MOB_0012.tiff
[1] MOB: 8. MOB_0015.tiff
[1] MOB: 9. MOB_0007.tiff
[1] MOB: 10. MOB_0023.tiff
[1] MOB: 11. MOB_0006.tiff
[1] MOB: 12. MOB_0022.tiff
[1] MOB: 13. MOB_0011.tiff
[1] MOB: 14. MOB_0020.tiff
[1] MOB: 15. MOB_0025.tiff
[1] MOB: 16. MOB_0024.tiff
[1] MOB: 17. MOB_0026.tiff
[1] MOB: 18. MOB_0010.tiff
[1] MOB: 19. MOB_0016.tiff
[1] MOB: 20. MOB_0003.tiff
[1] MOB: 21. MOB_0018.tiff
[1] MOB: 22. MOB_0008.ti

[3] NGS: 303. NGS_2504.tiff
[3] NGS: 304. NGS_0601.tiff
[3] NGS: 305. NGS_0930.tiff
[3] NGS: 306. NGS_5942.tiff
[3] NGS: 307. NGS_6183.tiff
[3] NGS: 308. NGS_6823.tiff
[3] NGS: 309. NGS_4005.tiff
[3] NGS: 310. NGS_8346.tiff
[3] NGS: 311. NGS_7281.tiff
[3] NGS: 312. NGS_1046.tiff
[3] NGS: 313. NGS_0673.tiff
[3] NGS: 314. NGS_4027.tiff
[3] NGS: 315. NGS_2261.tiff
[3] NGS: 316. NGS_7779.tiff
[3] NGS: 317. NGS_6330.tiff
[3] NGS: 318. NGS_4420.tiff
[3] NGS: 319. NGS_2711.tiff
[3] NGS: 320. NGS_5470.tiff
[3] NGS: 321. NGS_5883.tiff
[3] NGS: 322. NGS_1135.tiff
[3] NGS: 323. NGS_1702.tiff
[3] NGS: 324. NGS_5192.tiff
[3] NGS: 325. NGS_2736.tiff
[3] NGS: 326. NGS_6497.tiff
[3] NGS: 327. NGS_0386.tiff
[3] NGS: 328. NGS_4216.tiff
[3] NGS: 329. NGS_5860.tiff
[3] NGS: 330. NGS_3488.tiff
[3] NGS: 331. NGS_4960.tiff
[3] NGS: 332. NGS_3609.tiff
[3] NGS: 333. NGS_3099.tiff
[3] NGS: 334. NGS_3940.tiff
[3] NGS: 335. NGS_0487.tiff
[3] NGS: 336. NGS_8450.tiff
[3] NGS: 337. NGS_0148.tiff
[3] NGS: 338. NGS_43

[3] NGS: 727. NGS_0605.tiff
[3] NGS: 728. NGS_7738.tiff
[3] NGS: 729. NGS_8443.tiff
[3] NGS: 730. NGS_6127.tiff
[3] NGS: 731. NGS_4179.tiff
[3] NGS: 732. NGS_7485.tiff
[3] NGS: 733. NGS_5379.tiff
[3] NGS: 734. NGS_6603.tiff
[3] NGS: 735. NGS_4850.tiff
[3] NGS: 736. NGS_6421.tiff
[3] NGS: 737. NGS_5423.tiff
[3] NGS: 738. NGS_0106.tiff
[3] NGS: 739. NGS_0969.tiff
[3] NGS: 740. NGS_4685.tiff
[3] NGS: 741. NGS_7007.tiff
[3] NGS: 742. NGS_8460.tiff
[3] NGS: 743. NGS_3713.tiff
[3] NGS: 744. NGS_1958.tiff
[3] NGS: 745. NGS_2262.tiff
[3] NGS: 746. NGS_7461.tiff
[3] NGS: 747. NGS_7203.tiff
[3] NGS: 748. NGS_6912.tiff
[3] NGS: 749. NGS_5596.tiff
[3] NGS: 750. NGS_3823.tiff
[3] NGS: 751. NGS_4696.tiff
[3] NGS: 752. NGS_3857.tiff
[3] NGS: 753. NGS_7725.tiff
[3] NGS: 754. NGS_4834.tiff
[3] NGS: 755. NGS_1120.tiff
[3] NGS: 756. NGS_5516.tiff
[3] NGS: 757. NGS_4358.tiff
[3] NGS: 758. NGS_0233.tiff
[3] NGS: 759. NGS_4924.tiff
[3] NGS: 760. NGS_1605.tiff
[3] NGS: 761. NGS_7228.tiff
[3] NGS: 762. NGS_82

[3] NGS: 1150. NGS_0558.tiff
[3] NGS: 1151. NGS_3030.tiff
[3] NGS: 1152. NGS_3594.tiff
[3] NGS: 1153. NGS_7957.tiff
[3] NGS: 1154. NGS_1553.tiff
[3] NGS: 1155. NGS_2811.tiff
[3] NGS: 1156. NGS_4404.tiff
[3] NGS: 1157. NGS_7713.tiff
[3] NGS: 1158. NGS_5105.tiff
[3] NGS: 1159. NGS_0472.tiff
[3] NGS: 1160. NGS_3870.tiff
[3] NGS: 1161. NGS_3362.tiff
[3] NGS: 1162. NGS_6393.tiff
[3] NGS: 1163. NGS_0027.tiff
[3] NGS: 1164. NGS_5550.tiff
[3] NGS: 1165. NGS_0822.tiff
[3] NGS: 1166. NGS_4638.tiff
[3] NGS: 1167. NGS_2600.tiff
[3] NGS: 1168. NGS_1819.tiff
[3] NGS: 1169. NGS_2521.tiff
[3] NGS: 1170. NGS_8106.tiff
[3] NGS: 1171. NGS_3508.tiff
[3] NGS: 1172. NGS_5547.tiff
[3] NGS: 1173. NGS_5309.tiff
[3] NGS: 1174. NGS_3406.tiff
[3] NGS: 1175. NGS_3119.tiff
[3] NGS: 1176. NGS_4724.tiff
[3] NGS: 1177. NGS_6341.tiff
[3] NGS: 1178. NGS_3494.tiff
[3] NGS: 1179. NGS_3559.tiff
[3] NGS: 1180. NGS_2571.tiff
[3] NGS: 1181. NGS_0914.tiff
[3] NGS: 1182. NGS_8236.tiff
[3] NGS: 1183. NGS_4042.tiff
[3] NGS: 1184.

...

[14] LYT: 2759. LYT_3770.tiff
[14] LYT: 2760. LYT_2761.tiff
[14] LYT: 2761. LYT_3204.tiff
[14] LYT: 2762. LYT_0450.tiff
[14] LYT: 2763. LYT_2108.tiff
[14] LYT: 2764. LYT_0974.tiff
[14] LYT: 2765. LYT_0618.tiff
[14] LYT: 2766. LYT_0211.tiff
[14] LYT: 2767. LYT_3275.tiff
[14] LYT: 2768. LYT_0423.tiff
[14] LYT: 2769. LYT_1685.tiff
[14] LYT: 2770. LYT_0383.tiff
[14] LYT: 2771. LYT_1485.tiff
[14] LYT: 2772. LYT_2956.tiff
[14] LYT: 2773. LYT_1691.tiff
[14] LYT: 2774. LYT_0896.tiff
[14] LYT: 2775. LYT_0360.tiff
[14] LYT: 2776. LYT_1118.tiff
[14] LYT: 2777. LYT_3935.tiff
[14] LYT: 2778. LYT_3028.tiff
[14] LYT: 2779. LYT_2672.tiff
[14] LYT: 2780. LYT_1189.tiff
[14] LYT: 2781. LYT_0031.tiff
[14] LYT: 2782. LYT_2916.tiff
[14] LYT: 2783. LYT_1317.tiff
[14] LYT: 2784. LYT_2569.tiff
[14] LYT: 2785. LYT_1286.tiff
[14] LYT: 2786. LYT_3926.tiff
[14] LYT: 2787. LYT_2917.tiff
[14] LYT: 2788. LYT_2970.tiff
[14] LYT: 2789. LYT_2071.tiff
[14] LYT: 2790. LYT_2299.tiff
[14] LYT: 2791. LYT_3041.tiff
[14] LYT: 

[14] LYT: 3040. LYT_2528.tiff
[14] LYT: 3041. LYT_0484.tiff
[14] LYT: 3042. LYT_3039.tiff
[14] LYT: 3043. LYT_1389.tiff
[14] LYT: 3044. LYT_1754.tiff
[14] LYT: 3045. LYT_0676.tiff
[14] LYT: 3046. LYT_0964.tiff
[14] LYT: 3047. LYT_0384.tiff
[14] LYT: 3048. LYT_2567.tiff
[14] LYT: 3049. LYT_1483.tiff
[14] LYT: 3050. LYT_0439.tiff
[14] LYT: 3051. LYT_2075.tiff
[14] LYT: 3052. LYT_3106.tiff
[14] LYT: 3053. LYT_1834.tiff
[14] LYT: 3054. LYT_1453.tiff
[14] LYT: 3055. LYT_0526.tiff
[14] LYT: 3056. LYT_2370.tiff
[14] LYT: 3057. LYT_0712.tiff
[14] LYT: 3058. LYT_0747.tiff
[14] LYT: 3059. LYT_1086.tiff
[14] LYT: 3060. LYT_0141.tiff
[14] LYT: 3061. LYT_3050.tiff
[14] LYT: 3062. LYT_1433.tiff
[14] LYT: 3063. LYT_3691.tiff
[14] LYT: 3064. LYT_1164.tiff
[14] LYT: 3065. LYT_0329.tiff
[14] LYT: 3066. LYT_2561.tiff
[14] LYT: 3067. LYT_3021.tiff
[14] LYT: 3068. LYT_1250.tiff
[14] LYT: 3069. LYT_1949.tiff
[14] LYT: 3070. LYT_1182.tiff
[14] LYT: 3071. LYT_0842.tiff
[14] LYT: 3072. LYT_2296.tiff
[14] LYT: 

[14] LYT: 3322. LYT_2687.tiff
[14] LYT: 3323. LYT_2400.tiff
[14] LYT: 3324. LYT_0899.tiff
[14] LYT: 3325. LYT_2488.tiff
[14] LYT: 3326. LYT_0863.tiff
[14] LYT: 3327. LYT_3044.tiff
[14] LYT: 3328. LYT_2792.tiff
[14] LYT: 3329. LYT_3372.tiff
[14] LYT: 3330. LYT_3164.tiff
[14] LYT: 3331. LYT_2474.tiff
[14] LYT: 3332. LYT_0319.tiff
[14] LYT: 3333. LYT_2387.tiff
[14] LYT: 3334. LYT_1345.tiff
[14] LYT: 3335. LYT_0090.tiff
[14] LYT: 3336. LYT_3141.tiff
[14] LYT: 3337. LYT_2276.tiff
[14] LYT: 3338. LYT_2253.tiff
[14] LYT: 3339. LYT_3785.tiff
[14] LYT: 3340. LYT_3876.tiff
[14] LYT: 3341. LYT_3519.tiff
[14] LYT: 3342. LYT_1842.tiff
[14] LYT: 3343. LYT_1336.tiff
[14] LYT: 3344. LYT_1513.tiff
[14] LYT: 3345. LYT_1903.tiff
[14] LYT: 3346. LYT_2281.tiff
[14] LYT: 3347. LYT_2089.tiff
[14] LYT: 3348. LYT_0904.tiff
[14] LYT: 3349. LYT_0480.tiff
[14] LYT: 3350. LYT_0340.tiff
[14] LYT: 3351. LYT_1096.tiff
[14] LYT: 3352. LYT_3234.tiff
[14] LYT: 3353. LYT_3271.tiff
[14] LYT: 3354. LYT_3674.tiff
[14] LYT: 

[14] LYT: 3604. LYT_2390.tiff
[14] LYT: 3605. LYT_0062.tiff
[14] LYT: 3606. LYT_3454.tiff
[14] LYT: 3607. LYT_1802.tiff
[14] LYT: 3608. LYT_2183.tiff
[14] LYT: 3609. LYT_3657.tiff
[14] LYT: 3610. LYT_2702.tiff
[14] LYT: 3611. LYT_2258.tiff
[14] LYT: 3612. LYT_1781.tiff
[14] LYT: 3613. LYT_0057.tiff
[14] LYT: 3614. LYT_3022.tiff
[14] LYT: 3615. LYT_3485.tiff
[14] LYT: 3616. LYT_0952.tiff
[14] LYT: 3617. LYT_1964.tiff
[14] LYT: 3618. LYT_2354.tiff
[14] LYT: 3619. LYT_1328.tiff
[14] LYT: 3620. LYT_1213.tiff
[14] LYT: 3621. LYT_2485.tiff
[14] LYT: 3622. LYT_2721.tiff
[14] LYT: 3623. LYT_0574.tiff
[14] LYT: 3624. LYT_0358.tiff
[14] LYT: 3625. LYT_0255.tiff
[14] LYT: 3626. LYT_0391.tiff
[14] LYT: 3627. LYT_3880.tiff
[14] LYT: 3628. LYT_2720.tiff
[14] LYT: 3629. LYT_0838.tiff
[14] LYT: 3630. LYT_1976.tiff
[14] LYT: 3631. LYT_1256.tiff
[14] LYT: 3632. LYT_3325.tiff
[14] LYT: 3633. LYT_0511.tiff
[14] LYT: 3634. LYT_0478.tiff
[14] LYT: 3635. LYT_0873.tiff
[14] LYT: 3636. LYT_3154.tiff
[14] LYT: 

[14] LYT: 3885. LYT_2982.tiff
[14] LYT: 3886. LYT_3717.tiff
[14] LYT: 3887. LYT_2536.tiff
[14] LYT: 3888. LYT_3812.tiff
[14] LYT: 3889. LYT_3621.tiff
[14] LYT: 3890. LYT_1699.tiff
[14] LYT: 3891. LYT_3793.tiff
[14] LYT: 3892. LYT_1128.tiff
[14] LYT: 3893. LYT_1601.tiff
[14] LYT: 3894. LYT_1972.tiff
[14] LYT: 3895. LYT_3245.tiff
[14] LYT: 3896. LYT_2509.tiff
[14] LYT: 3897. LYT_1732.tiff
[14] LYT: 3898. LYT_3392.tiff
[14] LYT: 3899. LYT_0771.tiff
[14] LYT: 3900. LYT_0346.tiff
[14] LYT: 3901. LYT_3903.tiff
[14] LYT: 3902. LYT_0861.tiff
[14] LYT: 3903. LYT_1707.tiff
[14] LYT: 3904. LYT_1634.tiff
[14] LYT: 3905. LYT_2228.tiff
[14] LYT: 3906. LYT_2609.tiff
[14] LYT: 3907. LYT_1586.tiff
[14] LYT: 3908. LYT_0750.tiff
[14] LYT: 3909. LYT_1719.tiff
[14] LYT: 3910. LYT_0088.tiff
[14] LYT: 3911. LYT_3357.tiff
[14] LYT: 3912. LYT_0130.tiff
[14] LYT: 3913. LYT_3464.tiff
[14] LYT: 3914. LYT_1648.tiff
[14] LYT: 3915. LYT_3652.tiff
[14] LYT: 3916. LYT_2845.tiff
[14] LYT: 3917. LYT_3210.tiff
[14] LYT: 

In [6]:
def data_distribution(dataset):
  for label in LABELS:
    label_code = LABEL_TO_CODE[label]
#     label_count = N_SAMPLES_PER_LABEL[label_code]
    label_count = len(dataset[label_code])
    print('{:<2s} - {:<25s} {:<10s} {:<10s}'.format(str(label_code),str(LABELS_ABREVIATIONS[label]),f"({label})",str(label_count)))

data_distribution(images)

0  - Basophil                  (BAS)      79        
1  - Erythroblast              (EBO)      78        
2  - Eosinophil                (EOS)      424       
3  - Smudge cell               (KSC)      15        
4  - Lymphocyte (atypical)     (LYA)      11        
5  - Lymphocyte (typical)      (LYT)      3937      
6  - Metamyelocyte             (MMZ)      15        
7  - Monoblast                 (MOB)      26        
8  - Monocyte                  (MON)      1789      
9  - Myelocyte                 (MYB)      42        
10 - Myeloblast                (MYO)      3268      
11 - Neutrophil (band)         (NGB)      109       
12 - Neutrophil (segmented)    (NGS)      8484      
13 - Promyelocyte (bilobled)   (PMB)      18        
14 - Promyelocyte              (PMO)      70        


In [7]:
secure_images = images

## Split data

In [8]:
images = secure_images

In [9]:
train_images = [[] for i in range(num_classes)]
val_images = [[] for i in range(num_classes)]
test_images = [[] for i in range(num_classes)]

In [10]:
# test images
for label in LABELS:
    label_code = LABEL_TO_CODE[label]
    test_images[label_code] = images[label_code][:2]
    del images[label_code][:2]
    
# validation images
for label in LABELS:
    label_code = LABEL_TO_CODE[label]
    val_images[label_code] = images[label_code][:4]
    del images[label_code][:4]

# train images
# validation images
for label in LABELS:
    label_code = LABEL_TO_CODE[label]
    print(label_code)
    train_images[label_code] = images[label_code]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [11]:
data_distribution(images)

0  - Basophil                  (BAS)      73        
1  - Erythroblast              (EBO)      72        
2  - Eosinophil                (EOS)      418       
3  - Smudge cell               (KSC)      9         
4  - Lymphocyte (atypical)     (LYA)      5         
5  - Lymphocyte (typical)      (LYT)      3931      
6  - Metamyelocyte             (MMZ)      9         
7  - Monoblast                 (MOB)      20        
8  - Monocyte                  (MON)      1783      
9  - Myelocyte                 (MYB)      36        
10 - Myeloblast                (MYO)      3262      
11 - Neutrophil (band)         (NGB)      103       
12 - Neutrophil (segmented)    (NGS)      8478      
13 - Promyelocyte (bilobled)   (PMB)      12        
14 - Promyelocyte              (PMO)      64        


## Save data

In [12]:
if (not os.path.isdir(SPLIT_BALANCED_PATH)): os.mkdir(SPLIT_BALANCED_PATH)
    
rsb_train_path = os.path.join(SPLIT_BALANCED_PATH, 'train')
if (not os.path.isdir(rsb_train_path)): os.mkdir(rsb_train_path)
rsb_validation_path = os.path.join(SPLIT_BALANCED_PATH, 'validation')
if (not os.path.isdir(rsb_validation_path)): os.mkdir(rsb_validation_path)
rsb_test_path = os.path.join(SPLIT_BALANCED_PATH, 'test')
if (not os.path.isdir(rsb_test_path)): os.mkdir(rsb_test_path)

In [27]:
for label in LABELS:
    label_code = LABEL_TO_CODE[label]
    
    # train
    label_train_path = os.path.join(rsb_train_path, label)
    if (not os.path.isdir(label_train_path)): os.mkdir(label_train_path)
    for i, img in enumerate(train_images[label_code]):
        img_path = os.path.join(label_train_path, f"TRAIN_{i}_{label}.bmp")
        im = PIL.Image.fromarray(img)
        im.save(img_path)

    # validation
    label_val_path = os.path.join(rsb_validation_path, label)
    if (not os.path.isdir(label_val_path)): os.mkdir(label_val_path)
    for i, img in enumerate(val_images[label_code]):
        img_path = os.path.join(label_val_path, f"VAL_{i}_{label}.bmp")
        im = PIL.Image.fromarray(img)
        im.save(img_path)

    # test
    label_test_path = os.path.join(rsb_test_path, label)
    if (not os.path.isdir(label_test_path)): os.mkdir(label_test_path)
    for i, img in enumerate(test_images[label_code]):
        img_path = os.path.join(label_test_path, f"TEST_{i}_{label}.bmp")
        im = PIL.Image.fromarray(img)
        im.save(img_path)

/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/BAS/TRAIN_0_BAS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/BAS/TRAIN_1_BAS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/BAS/TRAIN_2_BAS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/BAS/TRAIN_3_BAS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/BAS/TRAIN_4_BAS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/BAS/TRAIN_5_BAS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/BAS/TRAIN_6_BAS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/BAS/TRAIN_7_BAS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/BAS/TRAIN_8_BAS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/BAS/TRAIN_9_BAS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/BAS/TRAIN_10_BAS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/BAS/TRAIN_11_BAS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/t

/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/MYO/TRAIN_2087_MYO.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/MYO/TRAIN_2088_MYO.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/MYO/TRAIN_2089_MYO.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/MYO/TRAIN_2090_MYO.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/MYO/TRAIN_2091_MYO.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/MYO/TRAIN_2092_MYO.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/MYO/TRAIN_2093_MYO.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/MYO/TRAIN_2094_MYO.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/MYO/TRAIN_2095_MYO.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/MYO/TRAIN_2096_MYO.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/MYO/TRAIN_2097_MYO.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/MYO/TRAIN_2098_MYO.bmp.npy
/home/jbonastre/

/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/NGS/TRAIN_7483_NGS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/NGS/TRAIN_7484_NGS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/NGS/TRAIN_7485_NGS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/NGS/TRAIN_7486_NGS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/NGS/TRAIN_7487_NGS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/NGS/TRAIN_7488_NGS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/NGS/TRAIN_7489_NGS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/NGS/TRAIN_7490_NGS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/NGS/TRAIN_7491_NGS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/NGS/TRAIN_7492_NGS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/NGS/TRAIN_7493_NGS.bmp.npy
/home/jbonastre/tfm/dataset/DatasetSplitBalanced/train/NGS/TRAIN_7494_NGS.bmp.npy
/home/jbonastre/

In [15]:
type(train_images[0][0])

numpy.ndarray